In [1]:
from ScraperHelper.Scrape import *
import pandas as pd

In [2]:
helper = driverHelper(logging_path="scrape.log")
helper.addOptions(
    arguments=["--incognito"],
    page_load_strategy="eager"
)
helper.forceCreateDriver( vpn_provider="nordvpn" )

12340-ERROR-There isn't any running driver
12340-INFO-Driver will perform option: --incognito
12340-INFO-Driver will have page_load_strategy: eager
12340-WARNING-There isn't any running driver to quit()
12340-WARNING-There isn't any running driver to quit()
12340-INFO-====== WebDriver manager ======
12340-INFO-Get LATEST chromedriver version for google-chrome 110.0.5481
12340-INFO-Driver [/home/durian/.wdm/drivers/chromedriver/linux64/110.0.5481/chromedriver] found in cache


<selenium.webdriver.chrome.webdriver.WebDriver (session="cd7f1730413635a840ff5f4947b6fbfe")>

In [3]:
search_url = "https://vtv.vn/tim-kiem.htm?keywords={}&page={}"
words_in_title =[
    "là",
    "và",
    "không"
]

In [4]:
all_articles = []
scraped_link = set()

In [5]:
for word in words_in_title:
    logging.info(f'Scraping search word {word}')
    page=1
    recent_article_link = ""
    while True:
        logging.info(f'Scraping page {page}')
        helper.forceGet(search_url.format(
            word, page
        ))
        
        logging.info("Getting articles info")
        scraped_articles = [
            {
                "link":ele.find_element(By.TAG_NAME, "a").get_attribute("href"), # link
                "title":ele.find_element(By.TAG_NAME, "a").get_attribute("title").strip(), # title
                "datetime":ele.find_element(By.CLASS_NAME, "time").get_attribute("innerHTML").strip(), # datetime
                "description":ele.find_element(By.CLASS_NAME, "sapo").get_attribute("innerHTML").strip(), #description
            }
            for ele in 
            helper.driver.find_elements(By.XPATH,"""//*[@id="SearchSolr1"]/li[*]""")
            # find all articles in a search page
            # each element found represents an article
        ]

        if recent_article_link == scraped_articles[0]["link"]:
            logging.info(f"Done scraping article from search word {word}")
            break
        else:
            recent_article_link = scraped_articles[0]["link"]

        for article in scraped_articles:
            if article["link"] not in scraped_link:
                all_articles.append(article)

        if page % 3 == 0:  
            pd.DataFrame(all_articles).to_feather("data/vtv_articles.feather")
        page+=1
    pd.DataFrame(all_articles).to_feather("data/vtv_articles.feather")

12340-INFO-Scraping search word là
12340-INFO-Scraping page 1
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=1
12340-INFO-Getting articles info
12340-INFO-Scraping page 2
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=2
12340-INFO-Getting articles info
12340-INFO-Scraping page 3
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=3
12340-INFO-Getting articles info
12340-INFO-Scraping page 4
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=4
12340-INFO-Getting articles info
12340-INFO-Scraping page 5
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=5
12340-INFO-Getting articles info
12340-INFO-Scraping page 6
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?keywords=là&page=6
12340-INFO-Getting articles info
12340-INFO-Scraping page 7
12340-INFO-Trying to reah the website https://vtv.vn/tim-kiem.htm?ke